# পর্ব ১২: এনক্রিপ্টেড ডেটা দিয়ে এনক্রিপ্টেড এনএন (NN) ট্রেইন করা (Train an Encrypted NN on Encrypted Data)
এই নোটবুকে আমরা নিউরাল নেটওয়ার্ক ট্রেইনিং ও প্রেডিকশানের ব্যাপারে এযাবত যা কিছু শিখেছি তার সমস্ত কিছু প্রয়োগ করে এনক্রিপ্টেড ডেটা দিয়ে এনক্রিপ্টেড মডেল ট্রেইন করব।

বিশেষত, আমরা আমাদের কাস্টম অটোগ্রাড ইঞ্জিন উপস্থাপন করব যা এনক্রিপ্টেড ডেটার উপর কাজ করে।

লেখকগণ:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Jason Paumier - Github: [@Jasopaum](https://github.com/Jasopaum)
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel)

অনুবাদক:

- Sourav Das - Twitter: [@adventuroussrv](https://twitter.com/adventuroussrv)
- Zarreen Reza - Twitter: [@zarreennreza](https://twitter.com/zarreennreza)

# ধাপ-১ঃ ওয়ার্কার ও খেলনা ডেটা তৈরি (Create Workers and Toy Data)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy

In [ ]:
# Set everything up
hook = sy.TorchHook(torch) 

alice = sy.VirtualWorker(id="alice", hook=hook)
bob = sy.VirtualWorker(id="bob", hook=hook)
james = sy.VirtualWorker(id="james", hook=hook)

In [ ]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
model = Net()

# ধাপ-২ঃ মডেল ও ডেটা এনক্রিপ্ট করণ (Encrypt the Model and Data)
Multi-Party Computation
এনক্রিপ্শান দুই ধাপে সম্পন্ন হয়। যেহেতু Secure Multi-Party Computation শুধুমাত্র পূর্ণসংখ্যার উপর কাজ করে, তাই দশমিকপূর্ণ সংখ্যার (যেমন weights ও activations) উপর কাজ করার জন্য  আমাদের আগে সকল সংখ্যাকে Fixed Precision দিয়ে এনকোড করতে হবে, যা আমাদের কয়েক দশমিক পর্যন্ত নির্ভুল মান দিবে। আমরা .fix_precision() কল করে এটা করতে পারি। 

তারপর আমরা অন্য ডেমোগুলোর ক্ষেত্রে যেভাবে করেছি, সেভাবে  .share() কল করে ডেটাগুলো Alice ও Bob এর মধ্যে পরস্পর শেয়ার করে এনক্রিপট করতে পারি। এখানে লক্ষণীয় যে, আমরা  requires_grad এর মান True সেট করে দিলে তা এনক্রিপ্টেড ডেটার জন্য একটি বিশেষ অটোগ্রেড মেথড যুক্ত করবে। স্বাভাবিকভাবেই, যেহেতু Secure Multi-Party Computation দশমিক সংখ্যার উপর কাজ করেনা, তাই এক্ষেত্রে সাধারণ PyTorch অটোগ্রেড মেথড করা যাবেনা। সুতরাং, আমাদের একটি বিশেষ AutogradTensor নোড যুক্ত করে backpropagation এর জন্য gradient graph গণনা করতে হবে। আপনি নিচের যেকোনো উপাদান প্রিন্ট করলে দেখবেন যে তাদের সাথে একটি AutogradTensor যুক্ত আছে।

In [ ]:
# We encode everything
data = data.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
target = target.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=james, requires_grad=True)

In [ ]:
print(data)

# ধাপ-৩ঃ ট্রেইনিং

এখন আমরা সাধারণ টেনসর লজিক ব্যবহার করে মডেল ট্রেইন করব।

In [ ]:
opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()

for iter in range(20):
    # 1) erase previous gradients (if they exist)
    opt.zero_grad()

    # 2) make a prediction
    pred = model(data)

    # 3) calculate how much we missed
    loss = ((pred - target)**2).sum()

    # 4) figure out which weights caused us to miss
    loss.backward()

    # 5) change those weights
    opt.step()

    # 6) print our progress
    print(loss.get().float_precision())

লস (loss) আসলেই কমেছে!

## fixed precision এর প্রভাব (Impact of fixed precision)
আপনার মনে হতে পারে যে সবকিছু এনক্রিপট করা কীভাবে হ্রাসমান লসের (loss) উপর প্রভাব ফেলছে। প্রকৃতপক্ষে, তাত্ত্বিক গণনা একই হওয়ায় প্রকৃত সংখ্যাগুলি এনক্রিপ্ট করা ট্রেইনিং এর খুব কাছাকাছি। আপনি এনক্রিপশন ছাড়াই এই একই মডেল চালিয়ে তা যাচাই করে দেখতে পারেন। এক্ষেত্রে আপনি __init__ এর সাহায্যে নিমক্ত মডেলের মতন নির্ণায়ক সূচনা (deterministic initialisation) নিশ্চিত করতে পারেন।:
```
with torch.no_grad():
    self.fc1.weight.set_(torch.tensor([[ 0.0738, -0.2109],[-0.1579,  0.3174]], requires_grad=True))
    self.fc1.bias.set_(torch.tensor([0.,0.1], requires_grad=True))
    self.fc2.weight.set_(torch.tensor([[-0.5368,  0.7050]], requires_grad=True))
    self.fc2.bias.set_(torch.tensor([-0.0343], requires_grad=True))
```

দশমিক সংখ্যাগুলোকে  fixed precision পূর্ণসংখ্যায় রূপান্তরিত করার ফলে মানের সামান্য তারতম্য দেখা যেতে পারে। সাধারণত   precision_fractional এর মান হচ্ছে ৩ এবং আপনি যদি তা ২ এ নামিয়ে আনেন তবে ক্লিয়ার টেক্সট ট্রেইনিং এর তারতম্য বেড়ে যাবে, অপরদিকে precision_fractional এর মান ৪ এ নিয়ে গেলে তা কমে যাবে। 

# অভিনন্দন - সম্প্রদায়ে যোগদানের এখনি সময়! (Congratulations!!! - Time to Join the Community!_

এই নোটবুক টিউটোরিয়ালটি সম্পন্ন করার জন্য অভিনন্দন! আপনি যদি এটি উপভোগ করে থাকেন এবং তথ্যর গোপনীয়তা রক্ষা পূর্বক, বিকেন্দ্রীভূত মালিকানাধিন এআই (AI) ও এআই (AI) সাপ্লাই চেইন (ডেটা) এর এই আন্দোলনকে সমর্থন করেন, নিম্নোক্ত উপায়ে আমাদের পাশে থাকতে পারেন!

### গিটহাবে PySyft কে স্টার দিন (Star PySyft on GitHub)

আমাদের সম্প্রদায়কে সাহায্য করার সবচেয়ে সহজ উপায় হল গিটহাব রিপোসিটোরি গুলোতে ষ্টার দেয়া।
 এটি আমরা যে অসাধারণ সরঞ্জামগুলি তৈরি করছি তার সচেতনতা বাড়াতে সহায়তা করে।

- [Star PySyft](https://github.com/OpenMined/PySyft)

### আমাদের স্ল্যাকে যোগ দিন (Join our Slack!)

সর্বশেষ আপডেট পাবার সর্বোত্তম উপায় হল আমাদের সম্প্রদায়ে যোগদান করা! এজন্য এই ফর্মটি পূরণ করতে পারেন।  [http://slack.openmined.org](http://slack.openmined.org)

### একটি কোড প্রকল্পে যোগদান করুন! (Join a Code Project!)

আমাদের সম্প্রদায়ে অবদান রাখার সর্বোত্তম উপায় হল কোড প্রকল্পে অবদান রাখা! আপনি PySyft Github Issues পেইজে গিয়ে ‘Projects’ এর জন্য ফিল্টার করতে পারেন। এটি আপনাকে শীর্ষস্থানীয় টিকেটগুলো দেখানোর মাধ্যমে আপনি কোন কোন প্রকল্পে কাজ করতে পারেন তার একটি ধারনা দিবে। যদি আপনি কোন প্রকল্পে যোগদান না করেই স্বল্প পর্যায়ের কোডিং করতে চান, তবে আপনি PySyft GitHub Issues পেইজে গিয়ে “Good First Issue” চিহ্নিত ইস্যুগুলো খুঁজে এক-কালিন কোন মিনি-প্রকল্প শুরু করতে পারেন!

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### দান করা (Donate)

আপনি যদি আমাদের কোডের প্রকল্পে অবদান রাখতে না পারেন কিন্তু তবুও আমাদের সমর্থন করতে চান, তবে আমাদের Open Collective এর Backer হতে পারেন। সকল অনুদান আমাদের ওয়েব হোসটিং এবং বিভিন্ন সাম্প্রদায়িক কার্যক্রম যেমন হ্যাকাথন, মিট-আপ ইত্যাদি কাজে ব্যয় হয়!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)